In [1]:
# Imports
import requests, json
import pandas as pd
import numpy as np
from pprint import pprint
import matplotlib.pyplot as plt
import seaborn as sns
import re
from pprint import pprint
from tqdm.auto import tqdm
from pulp import *

tqdm.pandas()

sns.set_style('whitegrid')
pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', 100)   

C:\Users\Bill\anaconda3\lib\site-packages\tqdm\std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
# base url for all FPL API endpoints
base_url = 'https://fantasy.premierleague.com/api/'

# get data from bootstrap-static endpoint
r = requests.get(base_url+'bootstrap-static/').json()

# show the top level fields
pprint(r, indent=2, depth=1, compact=True)

{ 'element_stats': [...],
  'element_types': [...],
  'elements': [...],
  'events': [...],
  'game_settings': {...},
  'phases': [...],
  'teams': [...],
  'total_players': 2939929}


In [3]:
# get player data from 'elements' field
players = r['elements']

# show data for first player
pprint(players[0])# create players dataframe
players = pd.json_normalize(r['elements'])

# show some information about first five players
players[['id', 'web_name', 'team', 'element_type']].head()

{'assists': 1,
 'bonus': 3,
 'bps': 292,
 'chance_of_playing_next_round': None,
 'chance_of_playing_this_round': None,
 'clean_sheets': 3,
 'code': 58822,
 'corners_and_indirect_freekicks_order': 2,
 'corners_and_indirect_freekicks_text': '',
 'cost_change_event': 0,
 'cost_change_event_fall': 0,
 'cost_change_start': 0,
 'cost_change_start_fall': 0,
 'creativity': '327.1',
 'creativity_rank': 113,
 'creativity_rank_type': 24,
 'direct_freekicks_order': 3,
 'direct_freekicks_text': '',
 'dreamteam_count': 0,
 'element_type': 2,
 'ep_next': '2.3',
 'ep_this': None,
 'event_points': 0,
 'first_name': 'Cédric',
 'form': '0.0',
 'goals_conceded': 27,
 'goals_scored': 1,
 'ict_index': '75.8',
 'ict_index_rank': 197,
 'ict_index_rank_type': 62,
 'id': 1,
 'in_dreamteam': False,
 'influence': '318.4',
 'influence_rank': 203,
 'influence_rank_type': 80,
 'minutes': 1481,
 'news': '',
 'news_added': None,
 'now_cost': 45,
 'own_goals': 0,
 'penalties_missed': 0,
 'penalties_order': None,
 'pena

,id,web_name,team,element_type
0,1,Cédric,1,2
1,2,Leno,1,1
2,3,Xhaka,1,3
3,4,Elneny,1,3
4,5,Holding,1,2


In [4]:
# create players dataframe
players = pd.json_normalize(r['elements'])

# Exctracting the teams
# create teams dataframe
teams = pd.json_normalize(r['teams'])

# Extracting the player positions

# get position information from 'element_types' field
positions = pd.json_normalize(r['element_types'])

# join players to teams
df = pd.merge(
    left=players,
    right=teams,
    left_on='team',
    right_on='id'
)

# join player positions
df = df.merge(
    positions,
    left_on='element_type',
    right_on='id'
)

# rename columns
df = df.rename(
    columns={'name':'team_name', 'singular_name':'position_name'}
)

# show result
df[
    ['first_name', 'second_name', 'team_name', 'position_name']
].head()

,first_name,second_name,team_name,position_name
0,Cédric,Alves Soares,Arsenal,Defender
1,Rob,Holding,Arsenal,Defender
2,Kieran,Tierney,Arsenal,Defender
3,Benjamin,White,Arsenal,Defender
4,Takehiro,Tomiyasu,Arsenal,Defender


In [ ]:
# Gameweek History for that season
# get data from 'element-summary/{PID}/' endpoint for PID=4
r = requests.get(base_url + 'element-summary/4/').json()

# show top-level fields for player summary
pprint(r, depth=1)

In [ ]:
def get_gameweek_history(player_id):
    '''get all gameweek info for a given player_id'''
    
    # send GET request to
    # https://fantasy.premierleague.com/api/element-summary/{PID}/
    r = requests.get(
            base_url + 'element-summary/' + str(player_id) + '/'
    ).json()
    
    # extract 'history' data from response into dataframe
    df = pd.json_normalize(r['history'])
    
    return df



def get_season_history(player_id):
    '''get all past season info for a given player_id'''
    
    # send GET request to
    # https://fantasy.premierleague.com/api/element-summary/{PID}/
    r = requests.get(
            base_url + 'element-summary/' + str(player_id) + '/'
    ).json()
    
    # extract 'history_past' data from response into dataframe
    df = pd.json_normalize(r['history_past'])
    
    return df


# # show a player gameweek history
# get_gameweek_history(7)[
#     [
#         'round',
#         'total_points',
#         'minutes',
#         'goals_scored',
#         'assists'
#     ]
# ].head()


# # show  a player gameweek history
# get_season_history(7)[
#     [
#         'season_name',
#         'total_points',
#         'minutes',
#         'goals_scored',
#         'assists',
#         'clean_sheets'
#     ]
# ].head(10)

**Data Frame combine the players teams and positions**

In [ ]:
# select columns of interest from players df
players = players[
    ['id', 'first_name', 'second_name', 'web_name', 'team',
     'element_type']
]

# join team name
players = players.merge(
    teams[['id', 'name']],
    left_on='team',
    right_on='id',
    suffixes=['_player', None]
).drop(['team', 'id'], axis=1)

# join player positions
players = players.merge(
    positions[['id', 'singular_name_short']],
    left_on='element_type',
    right_on='id'
).drop(['element_type', 'id'], axis=1)

players.head()

# Get the gameweek history of players

In [ ]:
# get gameweek histories for each player
points = players['id_player'].progress_apply(get_gameweek_history)

# combine results into single dataframe
points = pd.concat(df for df in points)

# join web_name
points = players[['id_player', 'web_name']].merge(
    points,
    left_on='id_player',
    right_on='element'
)

# The top 10 points scorers

In [ ]:
# get top scoring players
points.groupby(
    ['element', 'web_name']
).agg(
    {'total_points':'sum', 'goals_scored':'sum', 'assists':'sum'}
).reset_index(
).sort_values(
    'total_points', ascending=False
).head()

# Loading the required whole data for prediction analysis

In [5]:
r = requests.get(base_url+'bootstrap-static/').json()
r.keys()

dict_keys(['events', 'game_settings', 'phases', 'teams', 'total_players', 'elements', 'element_stats', 'element_types'])

In [6]:
# Loading the json data and making the dataframe
data = r

player_data_json = data['elements']
pdata = pd.json_normalize(player_data_json)
to_drop = ['chance_of_playing_this_round','chance_of_playing_next_round','code','cost_change_event','cost_change_event_fall','cost_change_start','cost_change_start_fall','dreamteam_count','ep_this','event_points','form','ict_index','in_dreamteam','news','photo','special','squad_number','status','transfers_in','transfers_in_event','transfers_out','transfers_out_event','value_form','value_season']
pdata.drop(to_drop, axis=1, inplace = True)
pdata['full_name'] = pdata.first_name + " " + pdata.second_name
pdata['element_type_name'] = pdata.element_type.map({x['id']:x['singular_name_short'] for x in data['element_types']})
pdata = pdata.loc[:,['full_name','first_name','second_name', 'element_type','element_type_name','id','team', 'team_code', 'web_name',
                     'saves','penalties_saved','clean_sheets','goals_conceded',
                     'bonus', 'bps','creativity','ep_next','influence', 'threat',
                     'goals_scored','assists','minutes', 'own_goals',
                     'yellow_cards', 'red_cards','penalties_missed',
                     'selected_by_percent', 'now_cost','points_per_game','total_points']]
pdata['team'] = pdata.team.map({x['id']:x['name'] for x in data['teams']})

In [7]:
df = pdata
df.head()

,full_name,first_name,second_name,element_type,element_type_name,id,team,team_code,web_name,saves,penalties_saved,clean_sheets,goals_conceded,bonus,bps,creativity,ep_next,influence,threat,goals_scored,assists,minutes,own_goals,yellow_cards,red_cards,penalties_missed,selected_by_percent,now_cost,points_per_game,total_points
0,Cédric Alves Soares,Cédric,Alves Soares,2,DEF,1,Arsenal,3,Cédric,0,0,3,27,3,292,327.1,2.3,318.4,111.0,1,1,1481,0,3,0,0,0.3,45,2.3,48
1,Bernd Leno,Bernd,Leno,1,GKP,2,Arsenal,3,Leno,10,0,1,9,0,69,0.0,2.7,85.0,0.0,0,0,360,0,0,0,0,0.6,45,2.5,10
2,Granit Xhaka,Granit,Xhaka,3,MID,3,Arsenal,3,Xhaka,0,0,6,38,3,334,522.9,2.0,395.4,262.0,1,2,2327,0,10,1,0,0.4,50,2.2,60
3,Mohamed Elneny,Mohamed,Elneny,3,MID,4,Arsenal,3,Elneny,0,0,0,17,0,156,131.5,1.5,163.2,49.0,0,2,801,0,1,0,0,1.7,45,1.9,27
4,Rob Holding,Rob,Holding,2,DEF,5,Arsenal,3,Holding,0,0,2,16,2,152,8.6,2.3,253.0,49.0,1,0,840,0,2,1,0,0.2,45,1.9,29


In [8]:
df.shape

(525, 30)

In [9]:
# Create the decision variables.. all the players
def create_dec_var(df):
    decision_variables = []
    
    for rownum, row in df.iterrows():
        variable = str('x' + str(rownum))
        variable = LpVariable(str(variable), lowBound = 0, upBound = 1, cat= 'Integer')
        decision_variables.append(variable)
                                  
    return decision_variables

# This is what we want to maximize (objective function)
def total_points(df,lst,prob):
    total_points = ""
    for rownum, row in df.iterrows():
        for i, player in enumerate(lst):
            if rownum == i:
                formula = row['total_points']*player
                total_points += formula

    prob += total_points
    
    return prob

# Add constraint for cash
def cash(df,lst,prob,avail_cash):
    total_paid = ""
    for rownum, row in df.iterrows():
        for i, player in enumerate(lst):
            if rownum == i:
                formula = row['now_cost']*player
                total_paid += formula
    prob += (total_paid <= avail_cash), "Cash"
    
    return prob

# Add constraint for number of goalkeepers
def team_gkp(df,lst,prob,avail_gk):
    total_gk = ""
    for rownum, row in df.iterrows():
        for i, player in enumerate(lst):
            if rownum == i:
                if row['element_type_name'] == 'GKP':
                    formula = 1*player
                    total_gk += formula

    prob += (total_gk == avail_gk), "GK"
    
    return prob

# Add constraint for number of defenders
def team_def(df,lst,prob,avail_def):
    total_def = ""
    for rownum, row in df.iterrows():
        for i, player in enumerate(lst):
            if rownum == i:
                if row['element_type_name'] == 'DEF':
                    formula = 1*player
                    total_def += formula

    prob += (total_def == avail_def), "DEF"
    
    return prob

# Add constraint for number of midfielders
def team_mid(df,lst,prob,avail_mid):
    total_mid = ""
    for rownum, row in df.iterrows():
        for i, player in enumerate(lst):
            if rownum == i:
                if row['element_type_name'] == 'MID':
                    formula = 1*player
                    total_mid += formula

    prob += (total_mid == avail_mid), "MID"
    
    return prob

# Add constraint for number of forwards
def team_fwd(df,lst,prob,avail_fwd):
    total_fwd = ""
    for rownum, row in df.iterrows():
        for i, player in enumerate(lst):
            if rownum == i:
                if row['element_type_name'] == 'FWD':
                    formula = 1*player
                    total_fwd += formula

    prob += (total_fwd == avail_fwd), "FWD"
    
    return prob

# Team Constraint for players should ne a max of 3 per team
def team_max_players(df,lst,prob):
    team_dict= {}
    for team in set(df.team_code):
        team_dict[str(team)]=dict()
        team_dict[str(team)]['avail'] = 3
        team_dict[str(team)]['total'] = ""
        for rownum, row in df.iterrows():
            for i, player in enumerate(lst):
                if rownum == i:
                    if row['team_code'] == team:
                        formula = 1*player
                        team_dict[str(team)]['total'] += formula

        prob += (team_dict[str(team)]['total'] <= team_dict[str(team)]['avail'])
    print(len(team_dict))

In [10]:
# The LpProblem
prob = LpProblem('FantasyTeam', LpMaximize)

In [11]:
# Assemble the whole problem data
def find_prob(df,ca,gk,de,mi,fw,prob):
    lst = create_dec_var(df)
    
    prob = total_points(df,lst,prob)
    prob = cash(df,lst,prob,ca)
    prob = team_gkp(df,lst,prob,gk)
    prob = team_def(df,lst,prob,de)
    prob = team_mid(df,lst,prob,mi)
    prob = team_fwd(df,lst,prob,fw)
    prob = team_max_players(df,lst,prob)
    
    return prob



In [12]:
# Find the optimal team
def df_decision(df,prob):
    variable_name = []
    variable_value = []

    for v in prob.variables():
        variable_name.append(v.name)
        variable_value.append(v.varValue)

    df_vals = pd.DataFrame({'variable': variable_name, 'value': variable_value})
    for rownum, row in df_vals.iterrows():
        value = re.findall(r'(\d+)', row['variable'])
        df_vals.loc[rownum, 'variable'] = int(value[0])

    df_vals = df_vals.sort_values(by='variable')

    #append results
    for rownum, row in df.iterrows():
        for results_rownum, results_row in df_vals.iterrows():
            if rownum == results_row['variable']:
                df.loc[rownum, 'Decision'] = results_row['value']

    return df

In [17]:
def LP_optimize(df, prob):
#     prob.writeLP('FantasyTeam2.lp')
    optimization_result = prob.solve()
    assert optimization_result == LpStatusOptimal
    print("Status:", LpStatus[prob.status])
    print("Optimal Solution to the problem: ", pulp.value(prob.objective))
    print ("Individual decision_variables: ")
    for v in prob.variables():
        print(v.name, "=", v.varValue)

First we want to see how it does for picking all 15 players on the team, with a maximum possible budget of 1000 (2 GK, 5 DEF, 5 FWD, 3 FWD)

In [ ]:
prob = find_prob(df,830,2,4,4,2,prob)

In [ ]:
df_final = df_decision(df,prob)
print(df_final[df_final['Decision']==1.0].now_cost.sum(), df_final[df_final['Decision']==1.0].total_points.sum())

In [ ]:
# The final 15
df_final[df_final['Decision']==1.0]


Now to see which formation maximizes the points earned by the starting XI.. Looking at the roster for the current season, the minimum price for a GK and a DEF are 40 each, and 45 for MID and FWD respectively. That means the maximum Cash we can use to pick a starting XI is 830.
7 possible formations::

    3-4-3
    3-5-2
    4-3-3
    4-4-2
    4-5-1
    5-3-2
    5-4-1



In [14]:
prob343 = find_prob(df,830,1,3,4,3,prob)
# prob352 = find_prob(df,830,1,3,5,2,prob)
# prob433 = find_prob(df,830,1,4,3,3,prob)
# prob442 = find_prob(df,830,1,4,4,2,prob)
# prob451 = find_prob(df,830,1,4,5,1,prob)
# prob532 = find_prob(df,830,1,5,3,2,prob)
# prob541 = find_prob(df,830,1,5,4,1,prob)

20


In [18]:
def prob_formations(df,prob):
    LP_optimize(df,prob)
    df_final = df_decision(df,prob)
    
    print(df_final[df_final['Decision']==1.0]['total_points'].sum())
    
    return(df_final[df_final['Decision']==1.0])

In [19]:
# 1. 3-4-3
prob_formations(df,prob343)

AttributeError: 'NoneType' object has no attribute 'solve'

In [ ]:
# 2. 3-5-2
prob_formations(df,prob352)

In [ ]:
# 3. 4-3-3
prob_formations(df,prob433)

In [ ]:
# 4. 4-4-2
prob_formations(df,prob442)

In [ ]:
# 5. 4-5-1
prob_formations(df,prob451)

In [ ]:
# 6. 5-3-2
prob_formations(df,prob532)

In [ ]:
# 7. 5-4-1
prob_formations(df,prob541)